In [107]:
import yfinance as yf
import numpy as np
import pandas as pd

tickers = {}
ticker_names = ["BTC-EUR", "SOL-EUR", "ETH-EUR", "GC=F", "SI=F"]
for ticker in ticker_names:
    tickers[ticker] = yf.download(ticker, start='2019-12-01', end='2024-11-08')
    tickers[ticker].reset_index(inplace=True)

print(tickers['BTC-EUR'])

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Price                       Date     Adj Close         Close          High  \
Ticker                                 BTC-EUR       BTC-EUR       BTC-EUR   
0      2019-12-01 00:00:00+00:00   6737.856934   6737.856934   6871.945801   
1      2019-12-02 00:00:00+00:00   6610.979004   6610.979004   6784.325195   
2      2019-12-03 00:00:00+00:00   6604.842773   6604.842773   6690.274902   
3      2019-12-04 00:00:00+00:00   6543.503418   6543.503418   6797.561035   
4      2019-12-05 00:00:00+00:00   6707.200684   6707.200684   6978.241211   
...                          ...           ...           ...           ...   
1799   2024-11-03 00:00:00+00:00  63717.242188  63717.242188  64272.445312   
1800   2024-11-04 00:00:00+00:00  63222.648438  63222.648438  63783.800781   
1801   2024-11-05 00:00:00+00:00  62362.378906  62362.378906  63730.976562   
1802   2024-11-06 00:00:00+00:00  63434.734375  63434.734375  64594.289062   
1803   2024-11-07 00:00:00+00:00  70484.195312  70484.195312  71

In [108]:
#Generate color palette from magma palette, excluding dark colors
from bokeh.palettes import Magma256
color_codes = [int(num) for num in np.linspace(100,255, len(ticker_names))]
colors = {k: v for k, v in zip(ticker_names, [Magma256[color] for color in color_codes])}
colors

{'BTC-EUR': '#892881',
 'SOL-EUR': '#C63C73',
 'ETH-EUR': '#F56C5B',
 'GC=F': '#FEB57C',
 'SI=F': '#FBFCBF'}

In [109]:
import numpy as np
from bokeh.core.properties import field
from bokeh.plotting import figure, output_file, show, column
from bokeh.io import curdoc
from bokeh.models import HoverTool, ColumnDataSource, RangeTool, VSpan, LabelSet, CustomJS

#Set dark theme to fit with magma palette
curdoc().theme='dark_minimal'

#The file to save the model
output_file("test.html")

#Instantiate figure object
dates = np.array(tickers['BTC-EUR']['Date'], dtype=np.datetime64)
graph = figure(height=500, width=1000, x_axis_type = "datetime", title = "Relative price changes", 
               tools='tap,xpan,reset', x_axis_location = "above", x_range =(dates[-365], dates[-1])) 

select = figure(title="Drag the middle and edges of the selection box to change the range",
                height=150, width=1000, y_range=graph.y_range, x_axis_type="datetime",
                tools="", toolbar_location=None)
range_tool = RangeTool(x_range=graph.x_range)
range_tool.overlay.fill_color = "#900C3F"
range_tool.overlay.fill_alpha = 0.2


#Names axes
graph.xaxis.axis_label = 'Date'
graph.yaxis.axis_label = '30-day price change (%)'

#Plotting the line graph

renderers = []

#Adding prices
for ticker_name in ticker_names:
    relativeChange = tickers[ticker_name].Close.pct_change(30)*100
    tickers[ticker_name]['RelativeChange'] = relativeChange
    tickers[ticker_name] = tickers[ticker_name].dropna() 
    tickers[ticker_name]['Legend_label'] = ticker_name

    source = ColumnDataSource(tickers[ticker_name])
    color = colors[ticker_name]
    l = graph.line('Date_', 'RelativeChange_', source=source, color=color, legend_label = ticker_name)
    renderers.append(l)

    select.line('Date_', 'RelativeChange_', source=source, color=color)

#Adding vspans for historical events
date_strings = ["2019-12-31", "2020-03-11", "2022-02-24", "2023-03-10", "2023-10-07", "2024-11-05"]
datetime_array = np.array(date_strings, dtype=np.datetime64)
label_strings = ["COVID-19 identified", "COVID-19 declared pandemic", "Russia-Ukraine War begins", "Silicon Valley Bank Collapse", "Israel-Hamas conflicts begin", "US Election Day"]

vspan_df = pd.DataFrame(dict(dates=datetime_array, width=np.full(len(datetime_array),2), labels=label_strings))
vspan_source = ColumnDataSource(vspan_df)

vspan = VSpan(x=field("dates"), line_width=field("width"), line_color="white", line_dash="dashed")

myMax = -10000
for ticker_name in tickers:
    currentMax = tickers[ticker_name]['RelativeChange'].max()
    if myMax < currentMax:
        myMax = currentMax


labels = LabelSet(x="dates", y=myMax-1, text="labels", x_offset=5, source=vspan_source, text_color="white")
graph.add_glyph(vspan_source, vspan)
select.add_glyph(vspan_source,vspan)
graph.add_layout(labels)

#Adding miscellaneous stylization
graph.legend.location = 'top_left'

hover = HoverTool(tooltips=[("Crypto name: ", "@Legend_label_"),("Date: ", "@Date_{%F}"), ("Price change: ", "@RelativeChange_{0.000}")],
                   renderers=renderers, formatters={'@Date_': 'datetime'})
graph.add_tools(hover)
select.ygrid.grid_line_color=None
select.add_tools(range_tool)

show(column(graph,select))

/tmp/ipykernel_46702/3564617701.py:14: UserWarning: no explicit representation of timezones available for np.datetime64
  dates = np.array(tickers['BTC-EUR']['Date'], dtype=np.datetime64)


Some sandboxing with how the RangeTool works. Will probably omit from the actual visualization

In [ ]:
# https://www.geeksforgeeks.org/adding-tooltips-to-a-timeseries-chart-hover-tool-in-python-bokeh/
import pandas as pd
import numpy as np
from bokeh.models import RangeTool, ColumnDataSource
from bokeh.plotting import figure, show, column
data = yf.download('SPY')
data.reset_index(inplace=True)

dates = np.array(data['Date'], dtype=np.datetime64)

data['RelativeChange'] = data.Close / data.Close.iloc[0]
source = ColumnDataSource(data)

p = figure(height=300, width=1000,  x_axis_type = "datetime", title = "Relative price changes", tools='xpan',
           toolbar_location = None, x_axis_location = "above", x_range =(dates[1500], dates[2500]))

p.line('Date_', 'RelativeChange_', source=source)
p.yaxis.axis_label="Relative Price"

select = figure(title="Drag the middle and edges of the selection box to change the range",
                height=150, width=1000, y_range=p.y_range, x_axis_type="datetime",
                tools="", toolbar_location=None)

range_tool = RangeTool(x_range=p.x_range)
range_tool.overlay.fill_color = "navy"
range_tool.overlay.fill_alpha = 0.2

select.line('Date_', 'RelativeChange_', source=source)

data2 = yf.download('NVDA')
source2 = ColumnDataSource(data2)
p.line('Date', 'Adj Close_NVDA', source=source2) 
select.line('Date', 'Adj Close_NVDA', source=source2)



select.ygrid.grid_line_color=None
select.add_tools(range_tool)
#select.toolbar.active_multi = range_tool

show(column(p,select))

In [ ]:
#TLDR: This doesn't work because of bokeh bullshit

#Long version:
#Apparently Python callbacks are only for noninteractive stuff
#Scaling the y axis would need to check all currently displayed stock data
#which is supposed to be interactive, but the number of displayed cryptos
#can change, and hence creation of a parameter list for the callback is impossible
#because JS callbacks can only get rendered elements as parameters, dictionaries are not allowed

#Once again, JS ruins everything
def update_y_range(attr, old, new):
    x_min = graph.x_range.start.astype(str)[:10]
    x_max = graph.x_range.end.astype(str)[:10]

    index_start = tickers['BTC-EUR'].index[tickers['BTC-EUR']['Date'] == x_min].tolist()[0]
    print(tickers['BTC-EUR']['Date'][0])
    print(index_start)
    index_end = tickers['BTC-EUR'].index[tickers['BTC-EUR']['Date'] == x_max].tolist()[0]

    myMin = 10000
    myMax = -10000
    for ticker_name in tickers:
        currentMax = tickers[ticker_name]['RelativeChange'][index_start:index_end+1].max()
        currentMin = tickers[ticker_name]['RelativeChange'][index_start:index_end+1].min()

        if myMax < currentMax:
            myMax = currentMax

        if myMin > currentMin:
            myMin = currentMin

    graph.y_range.start = myMin-1
    graph.y_range.end = myMax+1
    labels.y=myMax-1
    print(f"start: {x_min}\tend: {x_max}\nMin: {myMin}\nMax: {myMax}")
